In [ ]:
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors

In [ ]:
books = pd.read_csv("/content/Books.csv",sep=";")
ratings = pd.read_csv("/content/Ratings.csv",sep=";")
users = pd.read_csv("/content/Users.csv",sep=";")

<ipython-input-20-72b17b701049>:3: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  users = pd.read_csv("/content/drive/MyDrive/books_rec/Users.csv",sep=";")


In [ ]:
print(books.shape)
print(ratings.shape)
print(users.shape)

(271379, 5)
(1149780, 3)
(278859, 2)


In [ ]:
books.head()

,ISBN,Title,Author,Year,Publisher
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton & Company


In [ ]:
ratings.head()

,User-ID,ISBN,Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [ ]:
users.head()

,User-ID,Age
0,1,NaN
1,2,18
2,3,NaN
3,4,17
4,5,NaN


In [ ]:
ratings["ISBN"] = ratings["ISBN"].apply(lambda x: x.strip().strip("\'").strip("\\").strip('\"').strip("\#").strip("("))
mask = users['User-ID'].str.isnumeric()
temp= users

In [ ]:
r_users = ratings.groupby('User-ID')
r_books = ratings.groupby('ISBN')
avg_r_users=r_users['Rating'].mean()
num_r_users=r_users['Rating'].count()
avg_r_books=r_books['Rating'].mean()
num_r_books=r_books['Rating'].count()

avg_r_users.name = "avg_rating"
num_r_users.name="N_ratings"
avg_r_books.name="avg_rating"
num_r_books.name="N_ratings"

In [ ]:
users=users.merge(num_r_users,left_on='User-ID',right_index=True)
users=users.merge(avg_r_users,left_on='User-ID',right_index=True)
books=books.merge(num_r_books,left_on="ISBN",right_index=True)
books=books.merge(avg_r_books,left_on="ISBN",right_index=True)

In [ ]:
print(books.head())
print()
print(users.head())

         ISBN                                              Title  \
0  0195153448                                Classical Mythology   
1  0002005018                                       Clara Callan   
2  0060973129                               Decision in Normandy   
3  0374157065  Flu: The Story of the Great Influenza Pandemic...   
4  0393045218                             The Mummies of Urumchi   

                 Author  Year                Publisher  N_ratings  avg_rating  
0    Mark P. O. Morford  2002  Oxford University Press          1    0.000000  
1  Richard Bruce Wright  2001    HarperFlamingo Canada         14    4.928571  
2          Carlo D'Este  1991          HarperPerennial          3    5.000000  
3      Gina Bari Kolata  1999     Farrar Straus Giroux         11    4.272727  
4       E. J. W. Barber  1999   W. W. Norton & Company          1    0.000000  

  User-ID  Age  N_ratings  avg_rating
1       2   18          1    0.000000
6       7  NaN          1    0.000

In [ ]:
print(f"Out of {users.shape[0]} users only {users['N_ratings'].gt(0).sum(axis=0)} rated at least 1 book.")
print(f"Only {users['N_ratings'].gt(1).sum(axis=0)} rated at least 2 books.")
print(f"Only {users['N_ratings'].gt(9).sum(axis=0)} rated at least 10 books.")
print(f"Most active user rated {users['N_ratings'].max()} books.")
print()
print(f"Out of {books.shape[0]} books only {books['N_ratings'].gt(0).sum(axis=0)} are rated at least once.")
print(f"Only {books['N_ratings'].gt(1).sum(axis=0)} have at least 2 ratings.")
print(f"Only {books['N_ratings'].gt(9).sum(axis=0)} have at least 10 ratings.")
print(f"Most rated book was rated {books['N_ratings'].max()} times.")

Out of 99053 users only 99053 rated at least 1 book.
Only 43385 rated at least 2 books.
Only 12306 rated at least 10 books.
Most active user rated 13602 books.

Out of 270171 books only 270171 are rated at least once.
Only 124513 have at least 2 ratings.
Only 17480 have at least 10 ratings.
Most rated book was rated 2502 times.


In [ ]:
books.loc[books["N_ratings"]>20].sort_values(by="avg_rating",ascending=False).head(10)

,ISBN,Title,Author,Year,Publisher,N_ratings,avg_rating
3028,1844262553,Free,Paul Vincent,2003,Upfront Publishing,54,7.962963
27557,8445071416,El Hobbit,J. R. R. Tolkien,1991,Minotauro,23,7.478261
1101,0060256672,Where the Sidewalk Ends : Poems and Drawings,Shel Silverstein,1974,HarperCollins,33,7.121212
16191,0060248025,Falling Up,Shel Silverstein,1996,HarperCollins,38,6.921053
1763,0395177111,The Hobbit (Leatherette Collector's Edition),J. R. R. Tolkien,1973,Houghton Mifflin Company,24,6.791667
10288,0553274325,Johnny Got His Gun,Dalton Trumbo,1983,Bantam Books,32,6.750000
15179,0836221362,It's A Magical World: A Calvin and Hobbes Coll...,Bill Watterson,1996,Andrews McMeel Publishing,28,6.642857
5432,0439064864,Harry Potter and the Chamber of Secrets (Book 2),J. K. Rowling,1999,Scholastic,170,6.611765
5431,0439139597,Harry Potter and the Goblet of Fire (Book 4),J. K. Rowling,2000,Scholastic,194,6.541237
21546,1563890119,"Preludes and Nocturnes (Sandman, Book 1)",Neil Gaiman,1993,DC Comics,27,6.481481


Content Based Filtering

In [ ]:
conbooks = books[books['N_ratings']>20].copy()
conbooks.head()

,ISBN,Title,Author,Year,Publisher,N_ratings,avg_rating
5,0399135782,The Kitchen God's Wife,Amy Tan,1991,Putnam Pub Group,33,4.212121
18,0440234743,The Testament,John Grisham,1999,Dell,422,3.085308
19,0452264464,Beloved (Plume Contemporary Fiction),Toni Morrison,1994,Plume,180,3.411111
21,1841721522,New Vegetarian: Bold and Beautiful Recipes for...,Celia Brooks Brown,2001,Ryland Peters & Small Ltd,37,2.000000
26,0971880107,Wild Animus,Rich Shapero,2004,Too Far,2502,1.019584


In [ ]:
conbooks['Title']=conbooks['Title'].apply(lambda x:str(x))
conbooks['Author']=conbooks['Author'].apply(lambda x:str(x))
conbooks['Publisher']=conbooks['Publisher'].apply(lambda x:str(x))
conbooks['Title']=conbooks['Title'].apply(lambda x:x.split())
conbooks['Author']=conbooks['Author'].apply(lambda x:x.split())
conbooks['Publisher']=conbooks['Publisher'].apply(lambda x:x.split())
conbooks.head()

,ISBN,Title,Author,Year,Publisher,N_ratings,avg_rating
5,0399135782,"[The, Kitchen, God's, Wife]","[Amy, Tan]",1991,"[Putnam, Pub, Group]",33,4.212121
18,0440234743,"[The, Testament]","[John, Grisham]",1999,[Dell],422,3.085308
19,0452264464,"[Beloved, (Plume, Contemporary, Fiction)]","[Toni, Morrison]",1994,[Plume],180,3.411111
21,1841721522,"[New, Vegetarian:, Bold, and, Beautiful, Recip...","[Celia, Brooks, Brown]",2001,"[Ryland, Peters, &, Small, Ltd]",37,2.000000
26,0971880107,"[Wild, Animus]","[Rich, Shapero]",2004,"[Too, Far]",2502,1.019584


In [ ]:
conbooks["tags"]=conbooks["Title"]+conbooks['Author']+conbooks["Publisher"]
conbooks=conbooks.drop(columns=['Author','Publisher','Year'])
conbooks.head()

,ISBN,Title,N_ratings,avg_rating,tags
5,0399135782,"[The, Kitchen, God's, Wife]",33,4.212121,"[The, Kitchen, God's, Wife, Amy, Tan, Putnam, ..."
18,0440234743,"[The, Testament]",422,3.085308,"[The, Testament, John, Grisham, Dell]"
19,0452264464,"[Beloved, (Plume, Contemporary, Fiction)]",180,3.411111,"[Beloved, (Plume, Contemporary, Fiction), Toni..."
21,1841721522,"[New, Vegetarian:, Bold, and, Beautiful, Recip...",37,2.000000,"[New, Vegetarian:, Bold, and, Beautiful, Recip..."
26,0971880107,"[Wild, Animus]",2502,1.019584,"[Wild, Animus, Rich, Shapero, Too, Far]"


In [ ]:
conbooks['tags'] = conbooks['tags'].apply(lambda x: " ".join(x))
conbooks['Title'] = conbooks['Title'].apply(lambda x: " ".join(x))
conbooks.head()

,ISBN,Title,N_ratings,avg_rating,tags
5,0399135782,The Kitchen God's Wife,33,4.212121,The Kitchen God's Wife Amy Tan Putnam Pub Group
18,0440234743,The Testament,422,3.085308,The Testament John Grisham Dell
19,0452264464,Beloved (Plume Contemporary Fiction),180,3.411111,Beloved (Plume Contemporary Fiction) Toni Morr...
21,1841721522,New Vegetarian: Bold and Beautiful Recipes for...,37,2.000000,New Vegetarian: Bold and Beautiful Recipes for...
26,0971880107,Wild Animus,2502,1.019584,Wild Animus Rich Shapero Too Far


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(stop_words='english')
vector = cv.fit_transform(conbooks['tags']).toarray()
vector.shape

(6863, 8123)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
similarity = cosine_similarity(vector)
similarity.shape

(6863, 6863)

In [ ]:
def recommend(book):
    index = conbooks[conbooks['Title'] == book].index[0]
    l=[]
    distances = sorted(list(enumerate(similarity[index])),reverse=True,key = lambda x: x[1])
    for i in distances[1:6]:
        print(books.iloc[i[0]].Title)


In [ ]:
recommend("The Cat in the Hat")

El paraÃ­so (Tiempos modernos)
Napalm & Silly Putty
Native American Postcolonial Psychology (Suny Series in Transpersonal and Humanistic Psychology)
The Flintstones in Viva Rock Las Vegas: A Junior Novelization
A Heartbreaking Work of Staggering Genius


In [ ]:
pd_matrix= pd.merge(books.loc[books["N_ratings"]>20,"ISBN"],ratings , how="left",left_on="ISBN",right_on="ISBN").drop_duplicates()
pd_matrix

,ISBN,User-ID,Rating
0,0399135782,8,0
1,0399135782,11676,9
2,0399135782,29526,9
3,0399135782,36836,0
4,0399135782,46398,9
...,...,...,...
377631,0152012397,240700,7
377632,0152012397,241078,8
377633,0152012397,256167,8
377634,0152012397,259286,8


Item based CF

In [ ]:
pd_matrix1=pd_matrix.pivot(index='ISBN',columns='User-ID',values='Rating').fillna(0).astype('int8')
pd_matrix1

User-ID,8,9,10,14,16,17,19,26,32,39,...,278831,278832,278836,278838,278843,278844,278846,278849,278851,278854
ISBN,,,,,,,,,,,,,,,,,,,,,
000649840X,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0006547834,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0006550576,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0006550789,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0007110928,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8845205118,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8845247414,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
884590184X,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
itemmatrix1=csr_matrix(pd_matrix1.values)
itemmatrix1

<6863x55168 sparse matrix of type '<class 'numpy.int8'>'
	with 147021 stored elements in Compressed Sparse Row format>

In [ ]:
n1=10
knn=NearestNeighbors(metric='cosine',n_neighbors=n1,n_jobs=1)
knn.fit(itemmatrix1)

NearestNeighbors(metric='cosine', n_jobs=1, n_neighbors=10)

In [ ]:
distances1 , indices1 = knn.kneighbors(itemmatrix1)


In [ ]:
book_index=np.where(pd_matrix1.index=="0786808551")[0][0]
book_index

5993

In [ ]:
print(f"because you like {books.loc[books['ISBN']==pd_matrix1.index[indices1[book_index][0]], 'Title'].values[0]} you may like \n")
for i in range(1,n1):
  print(f"{books.loc[books['ISBN']==pd_matrix1.index[indices1[book_index][i]],'Title'].values[0]} with a similarity {distances1[book_index][i]:.3f} ")

because you like The Arctic Incident (Artemis Fowl, Book 2) you may like 

The Eternity Code (Artemis Fowl, Book 3) with a similarity 0.715 
Artemis Fowl (Artemis Fowl, Book 1) with a similarity 0.763 
Artemis Fowl (Artemis Fowl, Book 1) with a similarity 0.832 
LUCKY with a similarity 0.868 
Cold Hit (Alexandra Cooper Mysteries (Paperback)) with a similarity 0.882 
The Twits with a similarity 0.897 
Incubus with a similarity 0.898 
Killing Orders with a similarity 0.901 
Angel with a similarity 0.904 


User based CF

In [ ]:
pd_matrix2=pd_matrix.pivot(index='User-ID',columns='ISBN',values='Rating').fillna(0).astype('int8')
pd_matrix2

ISBN,000649840X,0006547834,0006550576,0006550789,0007110928,0007141076,0007154615,0020198817,0020198906,0020199600,...,880781210X,8807813025,8817106100,8817106259,8817131628,8845205118,8845247414,884590184X,8885989403,950491036X
User-ID,,,,,,,,,,,,,,,,,,,,,
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
14,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
16,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
278844,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
278846,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
278849,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
itemmatrix2=csr_matrix(pd_matrix2.values)
itemmatrix2

<55168x6863 sparse matrix of type '<class 'numpy.int8'>'
	with 147021 stored elements in Compressed Sparse Row format>

In [ ]:
n2=3
knn=NearestNeighbors(metric='cosine',n_neighbors=n2,n_jobs=1)
knn.fit(itemmatrix2)

NearestNeighbors(metric='cosine', n_jobs=1, n_neighbors=3)

In [ ]:
distances2 , indices2 = knn.kneighbors(itemmatrix2)


In [ ]:
uind=pd_matrix2.index.get_loc(99)
uind

24

In [ ]:
print(f" {users.loc[users['User-ID']==pd_matrix2.index[indices2[uind][0]]]}  \n")
print(f"users books - ")
uc=users.loc[users['User-ID']==pd_matrix2.index[indices2[uind][0]],'User-ID']
uc_isbn = ratings.loc[ratings['User-ID']==uc.iloc[0]].sort_values('Rating',ascending=False)['ISBN']
for i in uc_isbn.head():
  print(books.loc[books['ISBN']==i,'Title'].iloc[0])
print()
for i in range(1,n2):
  print(f"similar to user-id: {users.loc[users['User-ID']==pd_matrix2.index[indices2[uind][i]],'User-ID'].iloc[0]} with a similarity {distances2[uind][i]:.3f} ")
  uc=users.loc[users['User-ID']==pd_matrix2.index[indices2[uind][i]],'User-ID']
  uc_isbn = ratings.loc[ratings['User-ID']==uc.iloc[0]].sort_values('Rating',ascending=False)['ISBN']
  for i in uc_isbn.head():
    print(books.loc[books['ISBN']==i,'Title'].iloc[0])
  print()

    User-ID Age  N_ratings  avg_rating
98      99  42         12    5.083333  

users books - 
Rich Dad, Poor Dad: What the Rich Teach Their Kids About Money--That the Poor and Middle Class Do Not!
Creating Wealth : Retire in Ten Years Using Allen's Seven Principles of Wealth!
McDonald's: Behind the Arches
Fast Women
Female Intelligence

similar to user-id: 17644 with a similarity 0.240 
Rich Dad, Poor Dad: What the Rich Teach Their Kids About Money--That the Poor and Middle Class Do Not!
White Oleander : A Novel (Oprah's Book Club)
The Everything After College Book; Real-World Advice for Surviving and Thriving on Your Own

similar to user-id: 23835 with a similarity 0.240 
Cashflow Quadrant: Rich Dad's Guide to Financial Freedom
By Way of Deception : The Making and Unmaking of a Mossad Officer
Rich Dad, Poor Dad: What the Rich Teach Their Kids About Money--That the Poor and Middle Class Do Not!
The Silicon Boys and Their Valley of Dreams
The Complete Middle East Cookbook

